In [1]:
import numpy as np
import pandas as pd

train_path = "X_train.csv"
label_path = "y_train.csv"
test_path = "X_test.csv"
samples_path = "samples.csv"

df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
labeldf = pd.read_csv(label_path)


labels = labeldf.values
data = df.values
test_data = test_df.values

data = data[:,1:]
test_data = test_data[:,1:]
labels = labels[:,1]


print(data.shape)
print(test_data.shape)
print(labels.shape)



(5117, 17813)
(3411, 17813)
(5117,)


In [2]:
def count_instances(labels):
    unique, counts = np.unique(labels,return_counts=True)
    return dict(zip(unique,counts))

class_counts = count_instances(labels)

print("Class 0 Counts: {}".format(class_counts[0]))
print("Class 1 Counts: {}".format(class_counts[1]))
print("Class 2 Counts: {}".format(class_counts[2]))
print("Class 3 Counts: {}".format(class_counts[3]))


(num_train, max_timesteps) = data.shape
(num_test, _) = test_data.shape

Class 0 Counts: 3030
Class 1 Counts: 443
Class 2 Counts: 1474
Class 3 Counts: 170


In [3]:
# dk why I did this, might end up useful
def min_len(data):
    # faster to search column wise
    wherenan = np.isnan(data)
    for j in range(data.shape[1]):
        if np.any(wherenan[:,j]):
            return j
    """
    min_len = data.shape[1]
    for i in range(data.shape[0]):
        cond = np.isnan(data[i,:])
        if np.any(cond):
            nan_index = np.where(cond)[0][0]
            if nan_index < min_len:
                min_len = nan_index
            
    return min_len
    """

def fft_zero_pad(data):
    zero_padded = np.nan_to_num(data)
    return np.fft.fft(zero_padded, axis=0)

# implementing below link
# https://dsp.stackexchange.com/questions/25435/dft-for-audio-classification-signals-of-different-lengths
def fft_mod_min(data):
    l = min_len(data)
    # a divisor of the column number
    split_size = data.shape[1]/379
    zero_padded = np.nan_to_num(data)
    splits = np.split(zero_padded,split_size, axis = 1)
    splits = np.array(splits)
    # caution: not sure if the order of rows gets messed up here
    # so that the labels correspond to the correct order
    splits = splits.transpose(1,0,2)
    sums = np.sum(splits, axis = 1)
    
    return np.fft.fft(sums,axis=0)

mod_min_fft_train = fft_mod_min(data)
zero_pad_fft_train = fft_zero_pad(data)

mod_min_fft_test = fft_mod_min(test_data)
zero_pad_fft_test = fft_zero_pad(test_data)

print("Mod_min FFt shape {}".format(mod_min_fft_train.shape))
print("Zero Padded FFt shape {}".format(zero_pad_fft_train.shape))



Mod_min FFt shape (5117, 379)
Zero Padded FFt shape (5117, 17813)


In [4]:
#ts, filtered, rpeaks, templates_ts,templates, hear_rate_ts, heart_rate = ecg.ecg(data)
print(num_train)
a = []
for i in range(3):
    a.append(i)
print(a)

5117
[0, 1, 2]


In [5]:
from biosppy.signals.ecg import ecg


def extract_heart(data):
    All_ts = []
    All_filtered = []
    All_rpeaks = []
    All_templates_ts = []
    All_templates = []
    All_heart_rate_ts = []
    All_heart_rate = []
    thrown_away = []
    nanplaces = np.isnan(data)
    for i in range(data.shape[0]):
        stop_index = np.argmax(nanplaces[i,:]==True) -1
        try:
            ts, filtered, rpeaks, templates_ts,templates, heart_rate_ts, heart_rate = ecg(data[i,:stop_index], show= False)
        except:
            print("Index {} was bad".format(i))
            thrown_away.append(i)
        """
        print(ts.shape)
        print(filtered.shape)
        print(rpeaks.shape)
        print(templates_ts.shape)
        print(templates.shape)
        print(heart_rate_ts.shape)
        print(heart_rate.shape)
        """
        All_ts.append(ts)
        All_filtered.append(filtered)
        All_rpeaks.append(rpeaks)
        All_templates_ts.append(templates_ts)
        All_templates.append(templates)
        All_heart_rate_ts.append(heart_rate_ts)
        All_heart_rate.append(heart_rate)
        
    return All_ts, All_filtered, All_rpeaks, All_templates_ts, All_templates, All_heart_rate_ts, All_heart_rate, thrown_away
        

ts, filtered, rpeaks, templates_ts,templates, heart_rate_ts, heart_rate, thrown_away= extract_heart(data)


Index 269 was bad
Index 629 was bad
Index 690 was bad
Index 1024 was bad
Index 1115 was bad
Index 2354 was bad
Index 2627 was bad
Index 3350 was bad
Index 3424 was bad
Index 3962 was bad
Index 4021 was bad


In [59]:
def make_numpy(heart_thing):
    
    heart_np = np.zeros((len(heart_thing),max_timesteps))
    for i in range(heart_np.shape[0]):
        nans = np.empty((1,max_timesteps-len(heart_thing[i])))
        nans[:] = np.nan
        vals = np.array(heart_thing[i]).reshape(1,len(heart_thing[i]))
        heart_np[i,:] = np.concatenate((vals,nans),axis=1)

ts = make_numpy(ts)
filtered = make_numpy(filtered)
rpeaks = make_numpy(rpeaks)
templates_ts = make_numpy(templates_ts)
#templates = make_numpy(templates)
heart_rate_ts = make_numpy(heart_rate_ts)
heart_rate = make_numpy(heart_rate)


TypeError: object of type 'NoneType' has no len()

In [6]:
import math


def make_folds(num_folds, data, labels):
    y_0 = np.argwhere(labels==0)
    y_1 = np.argwhere(labels==1)
    y_2 = np.argwhere(labels==2)
    y_3 = np.argwhere(labels==3)

    X_0 = data[y_0].reshape(y_0.shape[0],data.shape[1])
    X_1 = data[y_1].reshape(y_1.shape[0],data.shape[1])
    X_2 = data[y_2].reshape(y_2.shape[0],data.shape[1])
    X_3 = data[y_3].reshape(y_3.shape[0],data.shape[1])



    num_0 = math.floor(X_0.shape[0]/num_folds)
    num_1 = math.floor(X_1.shape[0]/num_folds)
    num_2 = math.floor(X_2.shape[0]/num_folds)
    num_3 = math.floor(X_3.shape[0]/num_folds)



    y_folds = []
    x_folds = []

    for i in range(num_folds):
    
        y0 = np.zeros((num_0,1))
        y1 = np.ones((num_1,1))
        y2 = np.ones((num_2,1))*2
        y3 = np.ones((num_3,1))*3
        y = np.concatenate((y0,y1,y2,y3))
        y = y.reshape(y.shape[0],1)
        y_folds.append(y)
        x0 = X_0[i*num_0:(i+1)*num_0,:]
        x1 = X_1[i*num_1:(i+1)*num_1,:]
        x2 = X_2[i*num_2:(i+1)*num_2,:]
        x3 = X_3[i*num_3:(i+1)*num_3,:]
        x = np.concatenate((x0,x1,x2,x3))
        x_folds.append(x)
        
    return x_folds, y_folds
    

x_folds, y_folds = make_folds(5, data, labels)
for i, y in enumerate(y_folds):
    print(y_folds[i].shape)
    print("Number of instances in fold {}".format(i))
    class_counts = count_instances(y_folds[i])
    print("Class 0 Counts: {}".format(class_counts[0]))
    print("Class 1 Counts: {}".format(class_counts[1]))
    print("Class 2 Counts: {}".format(class_counts[2]))
    print("Class 3 Counts: {}".format(class_counts[3]))


    

(1022, 1)
Number of instances in fold 0
Class 0 Counts: 606
Class 1 Counts: 88
Class 2 Counts: 294
Class 3 Counts: 34
(1022, 1)
Number of instances in fold 1
Class 0 Counts: 606
Class 1 Counts: 88
Class 2 Counts: 294
Class 3 Counts: 34
(1022, 1)
Number of instances in fold 2
Class 0 Counts: 606
Class 1 Counts: 88
Class 2 Counts: 294
Class 3 Counts: 34
(1022, 1)
Number of instances in fold 3
Class 0 Counts: 606
Class 1 Counts: 88
Class 2 Counts: 294
Class 3 Counts: 34
(1022, 1)
Number of instances in fold 4
Class 0 Counts: 606
Class 1 Counts: 88
Class 2 Counts: 294
Class 3 Counts: 34


In [7]:
x_modtrain, y_modtrain = make_folds(5,mod_min_fft_train,labels)

x_zerotrain, y_zerotrain = make_folds(5,zero_pad_fft_train,labels)


